# Locais de referência para documentação:

https://ccxt.readthedocs.io/en/latest/manual.html

https://github.com/ccxt/ccxt/issues/8498



# 1 . Carregas as bibliotecas e as chaves para acesso via API

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
!pip3 install ccxt

In [ ]:
#as chaves de acesso estão presentes no arquivo keys.py
from google.colab import files
import keys
from keys import var_apiKey,var_secret

In [ ]:
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# CCXT é a biblioteca que contém as funções de comunicação via API
import asyncio
import ccxt.async_support as ccxt  # noqa: E402

# 2. Cria a instância da corretora FTX

In [ ]:
# account poderia ser um vetor com outras contas.
account = {'exchange_id': 'ftx', 'params': {'id': 'FTX', 'apiKey': var_apiKey, 'secret': var_secret}}
exchange_class = getattr(ccxt, account['exchange_id'])
exchange = exchange_class(account['params'])

#3. Funções para carregar a Carteira

In [ ]:
async def fetch_my_balance():
        balance = await exchange.fetch_balance()
        await exchange.close()
        carteira = balance
        return(carteira)

In [ ]:
# já cria o df e manipula as colunas
def carrega_carteira():
#  carteira = exchange.fetch_balance ()
  carteira = asyncio.get_event_loop().run_until_complete(fetch_my_balance())
  dfcarteira = pd.DataFrame(data = carteira['info']['result']) #, columns=('Moeda','Saldo','Saldo Disponivel','Valor em USD'))
  dfcarteira = dfcarteira[['coin','total','availableWithoutBorrow','usdValue']]
  dfcarteira.columns=('Moeda','Saldo','Saldo Disponivel','Valor em USD')

  dfcarteira.Saldo = dfcarteira.Saldo.astype(float)
  dfcarteira['Saldo Disponivel'] = dfcarteira['Saldo Disponivel'].astype(float)
  dfcarteira['Valor em USD'] = dfcarteira['Valor em USD'].astype(float)

  dfcarteira = dfcarteira[dfcarteira['Saldo'] != 0]

  dfcarteira['Valor Investido'] = int(0)
  dfcarteira['Lucro'] = int(0)
  dfcarteira['Percentual'] = int(0)
  dfcarteira['Valor Compra'] = int(0)
  dfcarteira['Valor Atual'] = int(0)

  return(dfcarteira)


#4. Funções para carregar todas as Trades e filtra as que não viraram ordens

---



In [ ]:
# chamada no modo async
async def fetch_my_trades():
        var_since = exchange.parse8601('2021-01-01T00:00:00Z')
        trades = await exchange.fetchMyTrades(since=var_since)
        await exchange.close()
        temp = trades
        return(temp)

In [ ]:
# carrega as ordens
def carrega_trades():


  all_trades = asyncio.get_event_loop().run_until_complete(fetch_my_trades())
  df_alltrades = pd.DataFrame(data = all_trades)

  # trata os dicionatios dentro do dicionario, criando novas colunas com essas chaves 
  # e populando-as com os valores

  dict_temp = df_alltrades.loc[0,'info']
  for k in dict_temp.keys():
    df_alltrades.loc[:,'info_'+k] = dict_temp[k]

  dict_temp = df_alltrades.loc[0,'fee']
  for k in dict_temp.keys():
    df_alltrades.loc[:,'fee_'+k] =  dict_temp[k]

  # cria o df das NEGOCIACOES QUE VIRARAM ordens,
  #retirando aquelas ORDERS que nao foram executadas

  df_trades = df_alltrades[~df_alltrades.order.isna()]
  df_trades = df_trades.assign(fator=0)
  df_trades = df_trades.assign(fatorquantidade=0)
  df_trades = df_trades.assign(fatorvalor=0)
  df_trades = df_trades.assign(quantacumulado=0)
  df_trades = df_trades.assign(valormedio=0)
  df_trades = df_trades.assign(valorinvestido=0)

  return(df_trades,df_alltrades)

#5. Funções para cálculos de médias

In [ ]:
## Funcao para criar valores que serão usados posteriormente para cálculo das médias
def calcaux(df):
  # separo o df desta moeda em duas partes: compra e venda

  # aqui vou tratar as informaçoes sobre vendas
  dfvenda = df.loc[df['side'] == 'sell']           # filtro somente o que é sell
  dfvenda['fator'] = int(-1)                       # a variácel fator receve -1, que simboliza venda
  dfvenda['fatorvalor'] = dfvenda['fator'] * dfvenda['price'] * dfvenda['amount']  # representa o montande em dollar do vendido
  dfvenda['fatorquantidade'] = dfvenda['fator'] * dfvenda['amount']                # representa a quantidade vendida
  # avgFillPrice => valor = price
  # filledSize   => quantidade = amount

  # aqui vou tratar as informaçoes sobre compras
  dfcompra = df.loc[df['side'] == 'buy'] # filtro somente o que é buy
  dfcompra['fator'] = int(1)
  dfcompra['fatorvalor'] = dfcompra['fator'] * dfcompra['price'] * dfcompra['amount']
  dfcompra['fatorquantidade'] = dfcompra['fator'] * dfcompra['amount']
  
  df = pd.concat([dfcompra, dfvenda])   # junto o df que fora dividido em compre e venda
  df.sort_values(['timestamp','id'], ascending=True,inplace=True)  # ordeno por data
  return(df)

In [ ]:
# funcao que calcula a média ponderada dos valor de compra
def calc_mediaponderada(df):

#8 side       (buy ou sell)
#9 price      valor unitário do token
#10 amount    quantidade
#11 cost      valor total = token * quantidade
#12 fee
#33 fator     1 para compra e -1 para venda
#34 fatorquantidade
#35 fatorvalor
#36 quantacumulado
#37 valormedio
#38 valorinvestido

  for x in range(len(df)):
    if df.iloc[x,8] == 'buy':
      df.iloc[x,33] =  1    # fator
    else:
      df.iloc[x,33] = -1    # fator


    df.iloc[x,34] = df.iloc[x,33]*df.iloc[x,10]       #34 fatorquantidade
    df.iloc[x,35] = df.iloc[x,33]*df.iloc[x,11]       #35 fatorvalor
    if x == 0:
      df.iloc[0,36] = df.iloc[0,33]*df.iloc[0,10]      # quantidade acumulada
      df.iloc[0,37] = df.iloc[0,33]*df.iloc[0,9]       # valor medio
    else:
      if df.iloc[x,8] == 'buy':
        df.iloc[x,36] = df.iloc[x-1,36] + df.iloc[x,10]           #36 quantacumulado
        df.iloc[x,37] =(df.iloc[x,9] * df.iloc[x,10]  + df.iloc[x-1,36] * df.iloc[x-1,37] ) / df.iloc[x,36] 
                       # valor atual * quanti atual + media * acumulado / total
        #print(df.iloc[x,37])
      else: # venda
        df.iloc[x,36] = df.iloc[x-1,36] - df.iloc[x,10]          #36 quantacumulado
        df.iloc[x,37] = df.iloc[x-1,37]          #37 valormedio  # quando vende, a media nao muda


    df.iloc[x,38] = df.iloc[x,36] * df.iloc[x,37]               # valor investido baseado no valor medio
    #print(f'vai limpar??: {df.iloc[x,38]}  =  {df.iloc[x,36]}  *  {df.iloc[x,37]}')
    if (df.iloc[x,38] < 0.1):             # quando quantidade em US$ é pequena, representa resíduo e na verdade vendeu tudo        
      df.iloc[x,36] = 0
      df.iloc[x,37] = 0 
      df.iloc[x,38] = 0 
      
  return(df)

In [ ]:
## Gera um arquivo com as informações presentes no dataframe
def geraarquivohistorico(df,nome):
#colocar no final as colunas que nao interessam

  cols_at_end = ['info', 'timestamp', 'id', 'order', 'type',
       'takerOrMaker', 'fee', 'info_id',
       'info_market', 'info_future', 'info_baseCurrency', 'info_quoteCurrency',
       'info_type', 'info_side', 'info_price', 'info_size', 'info_orderId',
       'info_time', 'info_tradeId', 'info_feeRate', 'info_fee',
       'info_feeCurrency', 'info_liquidity', 'fee_cost', 'fee_currency',
       'fee_rate', 'fator', 'fatorquantidade', 'fatorvalor']

  #cols_at_end = ['id', 'clientId','status','fator']
  df = df[[c for c in df if c not in cols_at_end] 
        + [c for c in cols_at_end if c in df]]
  #df.head()
  df = df.sort_values('timestamp',ascending=False)
  df.to_csv(nome, header=True,sep=';',decimal=',',index=False)

In [ ]:
## acrescenta /USD naquelas que nao sao tipo PERP,  LOCKED nem USD
def nome_completo(acao):
  if 'LOCKED' in acao:
    return(acao)
  elif 'PERP' in acao:
    return(acao)
  elif 'USD' in acao:
    return(acao)
  else:
    return(acao + '/USD')

In [ ]:
# coloca no dataframe carteira as informações de média que foram previamente calculadas e presentes no dataframa dados_brutos
# retorna a carteira com os valores de perda e genho
def acrescenta_media(dfdados,dfcarteira):
  valor_medio = 0
  for i in range(len(dfcarteira)):            # loop para cada item na carteira
    acao = dfcarteira.iloc[i]['Moeda']
    acao = nome_completo(acao)           # acrescendo o sufixo no nome do token
    #print(f'2 {acao}')
    try:
      # o valor medio de compra é pego no df trades, pegando o campo valormedio do registro mais recente
      valor_medio = float(dfdados[dfdados['timestamp'] == dfdados[dfdados['symbol']==acao].timestamp.max()].valormedio)
      # para o valor de marcado, é usado novamete a API e pego o valor instantaneo 
      valor_mercado =  exchange.markets[acao]['info']['price'] 
      flag = 1
    except:

      valor_medio = 0
      valor_mercado = 0
      flag = 0

    dfcarteira.iloc[i,4] = round(valor_medio  * dfcarteira.iloc[i]['Saldo'],2)           # 4 é o montante investido
    dfcarteira.iloc[i,5] = round(dfcarteira.iloc[i,3] - dfcarteira.iloc[i,4],2)               # 5 é o lucro, sendo 3 o montante corrente
    if flag:
      if (dfcarteira.iloc[i,4] > dfcarteira.iloc[i,3]):
      # deu lucro
        dfcarteira.iloc[i,6] = f'{(dfcarteira.iloc[i,3]/dfcarteira.iloc[i,4] - 1 ) * 100:.2f} %'
      else:
      # deu prejuizo
        dfcarteira.iloc[i,6] = f'{-(1 - dfcarteira.iloc[i,3]/dfcarteira.iloc[i,4]) * 100:.2f} %'

      dfcarteira.iloc[i,7] = valor_medio
      dfcarteira.iloc[i,8] = valor_mercado
    #print(valor_mercado)
    #print(acao,dfcarteira.iloc[i]['Valor Investido'])
  #print(saldo['Valor Investido'])
  return(dfcarteira)

#6.  Funções para carregar as ordens abertas (FecthOpenOrders) para ordens NORMAIS


In [ ]:
# chamada no modo async
async def fetch_my_openorders():
  exchange.options['fetchOpenOrders']['method'] = None
  ordens = await exchange.fetchOpenOrders()
  await exchange.close()
  temp = ordens
  return(temp)

In [ ]:
def carrega_ordens():
  df_openorderssummary = pd.DataFrame()

  open_orders = asyncio.get_event_loop().run_until_complete(fetch_my_openorders())

  if len(open_orders) > 0 :
    df_openorders = pd.DataFrame(data = open_orders)
    dict_temp = df_openorders.loc[0,'info']
  
    # trata os dicionatios dentro do dicionario, criando novas colunas com essas chaves e populando-as com os valores
    for k in dict_temp.keys():
      df_openorders.loc[:,'info_'+k] = dict_temp[k]

    df_openorderssummary = pd.DataFrame()
    df_openorderssummary[['datetime','symbol','side','price','amount','cost']] = df_openorders[['datetime','symbol','side','price','amount','cost']]
    df_openorderssummary['tipo'] = 'normal'
    df_openorderssummary.cost = df_openorderssummary.price * df_openorderssummary.amount

  return(df_openorderssummary)

#7.  Funções para carregar as ordens abertas (fetchOpenOrders {PARAM}) para ordens GATILHO

In [ ]:
# chamada no modo async
async def fetch_my_trigerorders():
  exchange.options['fetchOpenOrders']['method'] = 'privateGetConditionalOrders'
  ordens = await exchange.fetchOpenOrders()
  await exchange.close()
  temp = ordens
  return(temp)

In [ ]:
# carrega as ordens tipo gatilho. É a mesma função que carrega as ordens tradicionais, mas com um parametro especial
def carrega_ordensgatilho():
  df_triggerorderssummary = pd.DataFrame()
  if exchange.has['fetchOpenOrders']:
    trigger_orders = asyncio.get_event_loop().run_until_complete(fetch_my_trigerorders())

  if len(trigger_orders) > 0 :
    ## JA TRANSFORMA LOGO EM DATAFRAME
    df_triggerorders = pd.DataFrame(data = trigger_orders)

    # trata os dicionatios dentro do dicionario, criando novas colunas com essas chaves e populando-as com os valores
    dict_temp = df_triggerorders.loc[0,'info']
    for k in dict_temp.keys():
      df_triggerorders.loc[:,'info_'+k] = dict_temp[k]


    df_triggerorderssummary[['datetime','symbol','side','price','amount','cost']] = df_triggerorders[['datetime','symbol','side','price','amount','cost']]
    df_triggerorderssummary['tipo'] = 'gatilho'
    df_triggerorderssummary.cost = df_triggerorderssummary.price * df_triggerorderssummary.amount
  return(df_triggerorderssummary)

#8. MODULO PRINCIPAL

## 8.1 Primeiro carrega todas as cotações

In [ ]:
# chamada no modo async
async def load_my_markets():
        trades = await exchange.load_markets(True)    # TRUE FORÇA A ATUALIZAR COM UMA NOVA CONSULTA
        await exchange.close()
        temp = trades
        return(temp)

##8.2 Inicia as variáveis (dataframes) e o processamento

In [ ]:
if __name__ == '__main__':
  df_dadosbrutos = pd.DataFrame()
  df_orders =      pd.DataFrame()
  df_carteira =    pd.DataFrame()
  df_trades =      pd.DataFrame()
  df_alltrades =   pd.DataFrame()

  markets = asyncio.get_event_loop().run_until_complete(load_my_markets())  # carrega todas as cotacoes correntes

  df_carteira = carrega_carteira()
  df_trades,df_alltrades = carrega_trades()

  #sao todas as acoes que eu ja tive alguma ordem executada
  listaacoes = df_trades[df_trades['amount'].notnull()].symbol
  listaacoes = list(set(listaacoes))  # remove as duplicadas

  # LOOP PARA CADA ACAO PRESENTE NA CARTEIRA
  for acao in listaacoes:

    #filtrar o df com a acao do loop
    dftemp = df_trades[df_trades['symbol'] == acao]

    #acrescenta as colunas auxiliares
    dftemp = calcaux(dftemp)
 
    #acrescenta as medias
    dftemp = calc_mediaponderada(dftemp) # nova versao. Era #dftemp = calcmedia(dftemp)

    #concatena resultado de cada acao em dadosbrutos
    df_dadosbrutos = pd.concat([df_dadosbrutos, dftemp])
  
    # termino com as médias acrescentadas no df dados brutos, onde estao todas as trades

  #colocar as informacoes em um arquivo
  geraarquivohistorico(df_dadosbrutos,nome='ordens.csv')


  # baseado nas médias presentes em dadosbrutos,
  # calcula os ganhos e perdas e coloca no dataframe carteira
  df_carteira = acrescenta_media(df_dadosbrutos,df_carteira)

  # Vou consultar as ordens em aberto e no final guardar em um arquivo

  df_openorderssummary    = carrega_ordens()
  df_triggerorderssummary = carrega_ordensgatilho()

  if len(df_openorderssummary) > 0:       # CASO TENHA ALGUMA ORDEM ABERTA
    df_orders = df_orders.append(df_openorderssummary)
    if len(df_triggerorderssummary) > 0:  # CASO TENHA ALGUMA ORDEM TRIGGER ABERTA
      df_orders = df_orders.append(df_triggerorderssummary)

    df_orders.sort_values(['symbol'],inplace=True, ascending=False)
    df_orders.to_csv("openorders.csv", header=True,sep=';',decimal=',',index=False)

  df_carteira.sort_values(['Lucro'],inplace=True, ascending=False)
  df_carteira.to_csv("carteira.csv", header=True,sep=';',decimal=',',index=False)

  df_alltrades.sort_values(['timestamp'],inplace=True, ascending=False)
  df_alltrades.to_csv("alltrades.csv", header=True,sep=';',decimal=',',index=False)

#9. Relatórios em tela

##9.1 Lista a Carteria ordenada pela coluna 'Lucro', acrescentando o saldo no cabeçalho.

O campo "Valor Compra" possui o valor medio de compra da moeda, informação está que é usada para calcular o lucro ou prejuízo que é declarado no IR

In [ ]:
#markets.keys()
#markets['BTC/USD'].keys()
markets['BTC/USD']['info']['price']

#markets['SRM/USD'].keys()
#markets['SRM/USD']['info']['price']


In [ ]:
temp = 'Valor (' + str(int(df_carteira['Valor em USD'].sum())) + ' USD)'
df_carteira.rename(columns={'Valor em USD' : temp},  inplace = True)
df_carteira

##9.2 Lista as Ordens abertas, indicando o tipo (normar ou gatinho)

In [ ]:
df_orders.head(100)

##9.3 Lista as informacões mais relevantes de transações sobre uma determinada moeda. Nesse exemplo BTC/USD

In [ ]:
df_dadosbrutos[df_dadosbrutos.symbol == 'SRM/USD'][['datetime','side','quantacumulado','price','amount','cost']].sort_values('datetime',ascending=False)
